In [1]:
import numpy as np
import gym
from numpy.random import choice
import random
from tensorbuilder.api import *
import tensorflow as tf

env = gym.make("FrozenLake-v0")

hdf5 not supported (please install/reinstall h5py)


[2017-01-26 17:20:39,173] Making new env: FrozenLake-v0


In [2]:
def next_action(actions):
    n = len(actions)
    return choice(n, p=actions)
    
def select_columns(tensor, indexes):
    idx = tf.stack((tf.range(tf.shape(indexes)[0]), indexes), 1)
    return tf.gather_nd(tensor, idx)

def discount(rewards, y):
    r_accum = 0.0
    gains = []
    for r in reversed(list(rewards)):
        r_accum = r + y * r_accum 
        gains.insert(0, r_accum)
        
    return gains

In [81]:
n_actions = env.action_space.n
n_states = env.observation_space.n
model_name = "deep-q.model"
model_path = "/models/" + model_name
y = 0.95

graph = tf.Graph()
with graph.as_default():
    with tf.device("cpu:0"):
        S = tf.placeholder(tf.int32, [None], name='s')
        A = tf.placeholder(tf.int32, [None], name='a')
        R = tf.placeholder(tf.float32, [None], name='r')
        V1 = tf.placeholder(tf.float32, [None], name='v1')
        LR = tf.placeholder(tf.float32, [], name='lr')

        ops = dict(trainable=True, weights_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01), biases_initializer=None) #tf.random_uniform_initializer(minval=0, maxval=0.01))

        with tf.variable_scope("Actor"):
            PS = Pipe(
                S,
                T.one_hot(n_states)
#                 .relu_layer(32, **ops)
                .softmax_layer(4, **ops)
            )
        PWs = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Actor")
        
        with tf.variable_scope("Critic"):
            V = Pipe(
                S,
                T.one_hot(n_states)
                .linear_layer(1, **ops),
                T[:, 0]
            )
        VWs = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Critic")
        
        
        PSA = select_columns(PS, A)
        
        Trainer = tf.train.GradientDescentOptimizer(LR)
        
        E = R + y * V1 - V
        
        LossActor = -tf.reduce_sum(tf.log(PSA) * E)
        UpdateActor = Trainer.minimize(LossActor, var_list=PWs)
        
        LossCritic = Pipe(E, tf.nn.l2_loss, tf.reduce_sum)
        UpdateCritic = Trainer.minimize(LossCritic, var_list=VWs)

        Writer = tf.summary.FileWriter('/logs/' +  model_name, graph=graph)
        Saver = tf.train.Saver()

In [85]:
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device("cpu:0"):
        sess.run(tf.global_variables_initializer())
    #     saver.restore(sess, model_path)

        r_total = 0
        k = 2000.0
        
        train_period = 1
        train_reps = 4

        for t in range(200000):
            lr = 0.5 * (k / (k + t))

            s = env.reset()
            
            if t % train_period == 0:
                ss =  []
                as_ = []
                rs =  []
                v1s = []
                
            rs_ep = []

            while True:
                #get policy
                ps = sess.run(PS, feed_dict={S: [s]})[0]
                
                #next action
                a = next_action(ps)

                #take step
                s1, r, done, info = env.step(a)
                r_total += r
                
                v1 = sess.run(V, feed_dict={S: [s1]})[0]
                
                #save variables
                ss.append(s), rs.append(r), as_.append(a), v1s.append(v1)
                
                #train
                sess.run(UpdateCritic, feed_dict={
                    S: [s], A: [a], R: [r], V1: [v1],
                    LR: lr
                })
                sess.run(UpdateActor, feed_dict={
                    S: [s], A: [a], R: [r], V1: [v1],
                    LR: lr
                })

                #update state
                s = s1

                if done:
                    break
            
#             rs += discount(rs_ep, y)
            
            #train
            if t % train_period == 0:
                for i in range(train_reps):
                    sess.run(UpdateCritic, feed_dict={
                        S: ss, A: as_, R: rs, V1: v1s,
                        LR: lr / 5
                    })
                    sess.run(UpdateActor, feed_dict={
                        S: ss, A: as_, R: rs, V1: v1s,
                        LR: lr / 5
                    })
                    

            if t % 500 == 0:
                print r_total, "of", 500, ", lr:", lr
                r_total = 0
                Saver.save(sess, model_path)


0.0 of 500 , lr: 0.5
14.0 of 500 , lr: 0.4
30.0 of 500 , lr: 0.333333333333
158.0 of 500 , lr: 0.285714285714
314.0 of 500 , lr: 0.25


KeyboardInterrupt: 

In [80]:
s = env.reset()
done = False
with tf.Session(graph=graph) as sess:
    Saver.restore(sess, model_path)
    
    for i in range(100):
        ps = sess.run(PS, feed_dict={S: [s]})[0]
        a = next_action(ps)
        s, r, done, info = env.step(a)
        env.render()
        print("")

        if done:
            print(r)
            break

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Right)

1.0
